# Week 06
## Images

### Setup

Run the following 2 cells to import all necessary libraries and helpers for this week's exercises

In [ ]:
!wget -q https://github.com/DM-GY-9103-2024F-H/9103-utils/raw/main/src/image_utils.py

In [ ]:
import matplotlib.pyplot as plt
from image_utils import blur, edges, make_image, open_image

## REVIEW:

### Load image file

In [ ]:
himg = open_image("./data/hog.jpg")
display(himg)

### Copy image

In [ ]:
same_img = himg.copy()
display(same_img)

### Update pixels in image

In [ ]:
fimg = open_image("./data/flowers.jpg")
himg.update_pixels(fimg.pixels)
display(himg)

### Create image from pixel list

In [ ]:
new_img = make_image(himg.pixels, fimg.size[0], fimg.size[1])
display(new_img)

### Save image

In [ ]:
himg.save("./data/newhog.jpg")

### Processing pixels

In [ ]:
himg = open_image("./data/hog.jpg")

newpxs = []
for r,g,b in himg.pixels:
  newpxs.append((b, r, g))

himg.update_pixels(newpxs)
display(himg)

### With comprehension

In [ ]:
himg = open_image("./data/hog.jpg")
newpxs = [(g, b, r) for r,g,b in himg.pixels]
himg.update_pixels(newpxs)
display(himg)

### RGB to grayscale

We can also remove colors by making the pixel have a single value equal to the average of its original RGB values.

$\displaystyle average = \frac{R + G + B}{3}$

This is a good way to estimate the luminosity of each pixel: brighter pixels will be white and darker pixels will be black.

In [ ]:
himg = open_image("./data/hog.jpg")

bwpxs = []
for r,g,b in himg.pixels:
  gval = (r + g + b) // 3
  bwpxs.append(gval)

himg.update_pixels(bwpxs)
display(himg)

### Non-homework assignment

<img src="./imgs/red-coat-filter.jpg" width="720px">

The logic could be something like: if pixel is red, keep it, otherwise turn into greyscale.

In [ ]:
fimg = open_image("./data/hog.jpg")

fpxs = []
for r,g,b in fimg.pixels:
  if (r - b) > 140 and (g - b) > 100:
    fpxs.append((r, g, b))
  else:
    gv = (r + g + b) // 3
    fpxs.append((gv, gv, gv))

fimg.update_pixels(fpxs)
display(fimg)

### Filtering by Color

Let's formalize what we mean by filtering and be a bit more precise with what we are trying to do.

Let's say we're working on a vegetation detector and we want to be able to separate the pixels that represent plants and flowers from pixels that represent animals and other things.

We can start by creating a filter to separate the green pixels from our original image.

This is different than looking at the `green` color channel, or removing the `red` and `blue` channels, or exaggerating the green pixels.

In order to filter pixels of a certain color we have to go through the pixels and measure how similar they are to the color we wish to separate.

There are many ways to define "similar" when working with colors, but to keep it simple, let's define a `color_distance()` function that calculates the [Euclidean distance](https://en.wikipedia.org/wiki/Euclidean_distance) between two colors:

$\displaystyle dist = \sqrt{\left(R_0 - R_1\right)^2 + \left(G_0 - G_1\right)^2 + \left(B_0 - B_1\right)^2}$

In [ ]:
def color_distance(c0, c1):
  return ((c0[0] - c1[0])**2 + (c0[1] - c1[1])**2 + (c0[2] - c1[2])**2) ** 0.5

### Removing pixels

Now that we have a function for measuring color similarity we can go through the pixels and remove the ones that are very different from the color we want to keep. We'll remove pixels by turning them black with RGB value (0, 0, 0).

Since we're making some pretty significant changes to our image, let's keep a copy of the original. We can do this by just calling the `copy()` member function of an image object.

In [ ]:
himg = open_image("./data/hog.jpg")
fimg = himg.copy()

keep_color = (20, 180, 20)
thold = 120

filtpxs = []
for r,g,b in himg.pixels:
  if color_distance((r, g, b), keep_color) < thold:
    filtpxs.append((r, g, b))
  else:
    filtpxs.append((0, 0, 0))

fimg.update_pixels(filtpxs)

display(himg)
display(fimg)

### Filter other colors

How can we filter the image to keep only the flowers? Or to keep only the hedgehog?

It might help to define a `filter_color()` function here that takes an image and a color to keep as inputs, and returns another image with just the kept pixels ... while keeping the original image intact.

In [ ]:
# TODO: filter image to keep only flowers then filter image to keep only hedgehog

def filter_color(img, keep_color, thold=150):
  # TODO: fill this in
  return img

### Vegetation Filter

Now, what we actually want is to separate the image into two images, one with the hedgehog and another with the vegetation.

There are a couple of ways to achieve this.

One way to get the vegetation pixels in this case is to remove the hedgehog from the original image. We can do this by first getting the hedgehog pixels and then subtracting them from the original image, pixel by pixel, component by component.

We can use the `zip()` function to iterate through both pixel arrays at the same time and subtract their components.

In [ ]:
himg = open_image("./data/hog.jpg")

hog_img = filter_color(himg, (200,200,200), 200)

veget_pxs = []
for (r0,g0,b0), (r1,g1,b1) in zip(himg.pixels, hog_img.pixels):
  veget_pxs.append((r0-r1, g0-g1, b0-b1))

himg.update_pixels(veget_pxs)
display(himg)

# 🤔

This is ok.

The other way to separate the vegetation is to filter the green pixels into an array, then filter the yellow pixels into another array, and then add them together, pixel by pixel, component by component.

This works because we made the pixels we don't want black, or (0, 0, 0), so we can add green or yellow pixels to black without changing their color.

In [ ]:
# TODO: get a pixel array for the green pixels
# TODO: get a pixel array for the yellow pixels

# TODO: add green and yellow pixel arrays
#       zip() can help


# 🎉

### Counting pixels by colors

Now that we can separate pixels by color, we could use it to create an automatic deforestation sensor by separating and counting green pixels, and calculating the percentage of green areas on images.

We could implement a separate function to count the number of non-black pixels in an image after it has been filtered, but since our `filter_color()` function above already goes through all of the pixels in an image and detects pixels of specific colors, we can just create a modified version of it that counts those pixels and returns the ratio relative to the total number of pixels, instead of returning a filtered image.

We can call this new function `color_ratio()` and it will take and image, a color and a threshold as parameters, like the `filter_color()` function.

In [ ]:
# TODO: create color_ratio() function

def color_ratio(img, keep_color, thold=150):
  # TODO: modify the content of the filter_color() function
  return 0

We can try it out on a couple of forest images inside the `data/` directory.

In [ ]:
fimg = open_image("./data/forest-00.jpg")
display(fimg)

ffimg = filter_color(fimg, (0,200,0), 180)
display(ffimg)

green_ratio = color_ratio(fimg, (0,200,0), 180)
print(f"green %: {round(100 * green_ratio, 3)}")

### Try it out on some other images

There are $6$ other aerial forest images in the `data/` directory. Run the green pixel count code on them and see if the results make sense.

In [ ]:
# TODO: count green pixels in forest images

Once we start doing image analysis it's good to be able to extract different kinds of information from our images in case we want to categorize them, filter them further or retrieve them from large databases later.

This is kind of equivalent to how we extracted amplitude and frequency features from audio files.

### Dominant Channel

One feature we can easily extract from our images is the average value of each of its channels along with the average luminosity value.

This can be used to give us some idea about the dominant color or tones in an images.

In [ ]:
himg = open_image("./data/hog.jpg")

# array with 4 0s
hog_rgbl_sum = 4 * [0]

for r,g,b in himg.pixels:
  l = (r + g + b) // 3
  hog_rgbl_sum[0] += r
  hog_rgbl_sum[1] += g
  hog_rgbl_sum[2] += b
  hog_rgbl_sum[3] += l

hog_rgbl_avg = [s // len(himg.pixels) for s in hog_rgbl_sum]

hog_rgbl_avg

We can see that both the `green` and `red` channels have average values above the average luminosity value.

This makes sense since the image has a lot of green pixels, and the `red` channel contributes to the yellow and white pixels.

### Repeat for other image

Get the average value for each channel of a different image.

Maybe create a function...

Does the result make sense?

In [ ]:
# TODO: get average channel value for other image

def get_channel_avgs(pxs):
  # TODO: fill this in
  return []

In [ ]:
# TODO: run function

### Color Histograms

Color histograms are another useful method of analyzing images and extracting information about their color.

A histogram is just a concise way of seeing how many pixels have particular channel values. We'll go through an image's pixel array and count how many pixels have a $0$ for the `red` channel, how many have a $1$, a $2$, $3$, etc... and then we'll repeat this for the `green` and `blue` channels and for luminance.


In [ ]:
himg = open_image("./data/hog.jpg")

# create 4 arrays (RGB + luminance) with 255 0s, one for each possible value a pixel component can have
hog_hist_cnts = [256 * [0] for p in range(4)]

for r,g,b in himg.pixels:
  l = (r + g + b) // 3
  hog_hist_cnts[0][r] += 1
  hog_hist_cnts[1][g] += 1
  hog_hist_cnts[2][b] += 1
  hog_hist_cnts[3][l] += 1

In [ ]:
plt.plot(hog_hist_cnts[0], 'r')
plt.plot(hog_hist_cnts[1], 'g')
plt.plot(hog_hist_cnts[2], 'b')
plt.plot(hog_hist_cnts[3], color='#aaa')
plt.ylim((0, 10000))
plt.show()

### Get histogram for flower image

Maybe create some functions ... that can be reused later...

In [ ]:
# TODO: repeat process and draw histogram for flowers picture

def get_hist_counts(pxs):
  # TODO: fill this in
  return []

def plot_hist(rgbl_cnt, ylim=None):
  # TODO: fill this in
  plt.plot()

In [ ]:
# TODO: test functions

### Comparing histograms

Even though histograms can't by themselves tell us about the content of an image, comparing the most-frequent channel values in different images can tell us something about the images.

Let's get the most frequent value of each channel.

We'll do this by sorting `(value, count)` pairs by `count`.

And, we can get `(value, count)` pairs for our lists by simplify using the `enumerate()` function since the `value` here is the same as the index for each list.

Let's create a function to do this.

In [ ]:
def get_most_frequent(hist_counts):
  # to sort by the second element of an array or tuple
  def byCount(A):
    return A[1]

  # sort (value, count) pairs by count for each of the 4 histogram lists
  sorted_rgbl_cnt = [sorted(enumerate(hl), key=byCount, reverse=True) for hl in hist_counts]

  # grab the first element of the first (value, count) pair of each list
  top_rgbl = [shl[0][0] for shl in sorted_rgbl_cnt]

  return top_rgbl

In [ ]:
get_most_frequent(hog_hist_cnts), get_most_frequent(flo_hist_cnts)

# 🤔

What this tells us is that the hedgehog image is generally darker (all channel values and luminance are less than $100$) and `green` is possibly the brightest channel.

The flower picture is bright (luminance above $200$) and there are more bright `red` tones than any other tones.

But besides these easily interpretable descriptions, there's a lot of information in this data that might not be obvious to us if we're only looking at $1$ or $2$ images.

A color histogram is another example of a transformation that encodes/compresses the information in the original data into less numbers, while keeping some significant information about it.

In this case, information about an image with $600,000$ pixels and $3$ channels ($1,800,000$ values) is being represented by $4$ lists of $256$ numbers.

### Repeat with other images

Plot histograms and get the average and most frequent channel values for $2$ other images.

In [ ]:
# TODO: repeat histogram for other images

### Edge Detection

We've looked at some techniques for getting color information from images, but images are more than just colors.

We might be interested in also quantifying the shapes and textures present in our images.

We can start by extracting the edges of shapes in our image. There are many ways of doing this, but the simplest way is to subtract our original image from a blurry version of it and threshold the result.

Since we are not so concerned with color at this point we should work with grayscale images.

Our overall algorithm will be something like:
- open an image
- make it black & white
- blur it
- subtract the blurry b&w pixels from the original b&w pixels
- threshold the result

Threshold means making slightly bright pixels really bright and all other pixels really dark.

Let's do this in steps:

#### Open an image and extract its pixels

We can use the `open_image()` function and the `pixels` member variable to open and extract pixels, and `display()` to show the image.

In [ ]:
# TODO: implement edge extraction algorithm
  # open an image and extract its pixels

mimg = ''''''
ipxs = ''''''

display(mimg)

#### Blur the image using `blur()`

And display it.

The `blur()` function takes an image object as a parameter and an optional second parameter that specifies the amount of blurring. It returns another image object.

Experiment with the parameter a little bit, but the default value is good for extracting edges.

Let's also get the pixels for the blurred image and display it with `display()`.

In [ ]:
# TODO: implement edge extraction algorithm
  # blur image with the blur() function

bimg = ''''''
bpxs = ''''''

display(bimg)

#### Make the images grayscale

We saw this a few cells back. We can average the `RGB` values to get a grey luminance value.

Get grayscale versions of the original image and the blurry image

Display the results.

In [ ]:
# TODO: implement edge extraction algorithm
  # make the image b&w

bwimg = mimg.copy()
bwbimg = bimg.copy()

# TODO: make b&w
bwpxs = ''''''
bwbpxs = ''''''

bwimg.update_pixels(bwpxs)
display(bwimg)

bwbimg.update_pixels(bwbpxs)
display(bwbimg)

#### Subtract the blurred pixels from the original pixel values

The `zip()` function might help iterate through the pixel arrays from both images at the same time.

Display the resulting image.

In [ ]:
# TODO: implement edge extraction algorithm
  # subtract blurry b&w image from original b&w image

simg = bwpxs.copy()

# TODO: subtract bwbpxs from bwpxs
spxs = ''''''

simg.update_pixels(spxs)
display(simg)

#### Threshold the resulting pixel values

We'll go through the array and check each pixel:<br>
if its luminance is greater than a threshold value, we'll make it $255$, otherwise we'll make it $0$.

We can start with a threshold value of $16$ and see what happens.

In [ ]:
# TODO: implement edge extraction algorithm
  # threshold pixel values

timg = simg.copy()

# TODO: threshold pixels
tpxs = ''''''

timg.update_pixels(tpxs)
display(timg)

Great !

### Let's repeat it for a different image

First, create a function that takes an image as a parameter and returns another image with edge information.

In [ ]:
# TODO: create edge extraction function

def edges(img, rad=1, thold=16):
  # TODO: fill this in
  return img

In [ ]:
# TODO: use function

### Count edges

It helps to have a single value that we can use to compare edge information between images.

Let's create a function that counts the number of white pixels in an edge-extraction image.

We'll divide this number by the number of pixels in the image to get a rough idea of how _edgy_ any image is.

In [ ]:
def edge_ratio(img, rad=1, thold=16):
  eimg = edges(img, rad=rad, thold=thold)
  sum255 = sum([1 for rgb in eimg.pixels if rgb[0] == 255])
  npxs = len(eimg.pixels)
  return round(sum255 / npxs, 4)

mimg = open_image("./data/hog.jpg")
display(edges(mimg, 2))
edge_ratio(mimg, 2)

### Count edges for different images

Do the results make sense? Why did we divide the sum by the total number of pixels?

In [ ]:
# TODO: run the edge_ratio() function on a few images